In [ ]:
!pip install tqdm scidownl

In [2]:

def download_multi_papers():
    """Example of downloading multiple papers.
    All papers will be downloaded to the ./paper/ directory,
    and their filenames are the paper titles.
    """
    source = [
        ("https://doi.org/10.1145/3375633", 'doi', "./paper/"),
        ("31395057", 'pmid', "./paper/"), 
        ("24686414", 'pmid', "./paper/"),
        ("Aggregated Residual Transformations for Deep Neural Networks", 'title', "./paper/"),
    ]
    for paper, paper_type, out in source:
        scihub_download(paper, paper_type=paper_type, out=out)

#("https://doi.org/10.1145/3375633", 'doi', "./paper/")
#three options 1. (doi,'doi',./export location) 2.(pmid, 'pmid', ./export location) 3.(title, 'title',./paper/)


In [3]:
download_multi_papers()

NameError: name 'scihub_download' is not defined

#### PDF Downloader

In [9]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm


In [15]:
df = pd.read_csv('output.csv')
doi_list = list(df['DOI'])
print(len(doi_list))

doi_list = [doi for doi in doi_list if pd.notna(doi)]
print(len(doi_list))

350
301


In [18]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

titles = []
citations = []
dois = []

def pdf_downloader(doi):
    doi = doi.replace(':', '/')
    base_url = "https://sci-hub.ru"
    url = f"{base_url}/{doi}"

    response = requests.get(url)
    response.raise_for_status()

    soup = BeautifulSoup(response.content, 'html.parser')

    try:
        citation_div = soup.find('div', {'id': 'citation'})
        title = citation_div.find('i').text
        citation = citation_div.text
        titles.append(title)
        citations.append(citation)
        dois.append(doi)
    except:
        titles.append(None)
        citations.append(None)
        dois.append(doi)
        return
    
    try:
        pdf_relative_url = soup.find('embed', {'type': 'application/pdf'}).get('src')
    except:
        return

    pdf_url = urljoin(base_url, pdf_relative_url)

    pdf_response = requests.get(pdf_url)

    filename = f"data/{doi.replace('/', ':')}.pdf"

    with open(filename, "wb") as file:
        file.write(pdf_response.content)

# Create a ThreadPoolExecutor with a maximum of 5 concurrent workers
with ThreadPoolExecutor(max_workers=5) as executor:
    # Submit tasks to the executor
    futures = [executor.submit(pdf_downloader, doi) for doi in doi_list]

    # Iterate over completed futures to check for any exceptions
    for future in tqdm(as_completed(futures), total=len(futures)):
        try:
            future.result()
        except Exception as e:
            print(f"An exception occurred: {str(e)}")


 29%|██▉       | 88/301 [00:08<00:35,  5.94it/s]

An exception occurred: 503 Server Error: Service Temporarily Unavailable for url: http://library.lol/main/7A001B5426DF1BEA8DD1BDA1CEEBC7AB


 31%|███       | 92/301 [00:09<00:44,  4.67it/s]

An exception occurred: 403 Client Error: Forbidden for url: https://sci-hub.ru/10.1016/j.chemosphere.2019.125646


 32%|███▏      | 97/301 [00:10<00:23,  8.76it/s]

...

An exception occurred: 403 Client Error: Forbidden for url: https://sci-hub.ru/10.3406/geo.1998.20847
An exception occurred: 403 Client Error: Forbidden for url: https://sci-hub.ru/10.1016/S1364-8152(98)00003-6
An exception occurred: 403 Client Error: Forbidden for url: https://sci-hub.ru/10.4141/S98-055
An exception occurred: 403 Client Error: Forbidden for url: https://sci-hub.ru/10.1007/BF02913341
An exception occurred: 403 Client Error: Forbidden for url: https://sci-hub.ru/10.2136/sssaj1996.03615995006000050035x
An exception occurred: 403 Client Error: Forbidden for url: https://sci-hub.ru/10.1007/bf02905197
An exception occurred: 403 Client Error: Forbidden for url: https://sci-hub.ru/10.1016/0022-1694(93)90127-U
An exception occurred: 403 Client Error: Forbidden for url: https://sci-hub.ru/10.2307/4002692
An exception occurred: 403 Client Error: Forbidden for url: https://sci-hub.ru/10.1080/10220119.1995.9647873
An exception occurred: 403 Client Error: Forbidden for url: https:/

 97%|█████████▋| 293/301 [00:14<00:00, 45.84it/s]

An exception occurred: 403 Client Error: Forbidden for url: https://sci-hub.ru/10.1080/00103629309368884
An exception occurred: 403 Client Error: Forbidden for url: https://sci-hub.ru/10.1007/s100400050026
An exception occurred: 403 Client Error: Forbidden for url: https://sci-hub.ru/10.1071/BT9910431
An exception occurred: 403 Client Error: Forbidden for url: https://sci-hub.ru/10.1016/0921-8181(91)90118-G
An exception occurred: 403 Client Error: Forbidden for url: https://sci-hub.ru/10.1016/0031-0182(91)90173-O
An exception occurred: 403 Client Error: Forbidden for url: https://sci-hub.ru/10.1016/0341-8162(88)90008-2
An exception occurred: 403 Client Error: Forbidden for url: https://sci-hub.ru/10.1080/00139157.1990.9931055
An exception occurred: 403 Client Error: Forbidden for url: https://sci-hub.ru/10.1111/j.1467-8470.1986.tb00512.x
An exception occurred: 403 Client Error: Forbidden for url: https://sci-hub.ru/10.1111/j.1475-2743.1987.tb00724.x
An exception occurred: 403 Client Er

100%|██████████| 301/301 [00:14<00:00, 20.30it/s]

An exception occurred: 403 Client Error: Forbidden for url: https://sci-hub.ru/10.1097/00010694-198301000-00010
An exception occurred: 403 Client Error: Forbidden for url: https://sci-hub.ru/10.1016/0016-7061(76)90069-0
An exception occurred: 403 Client Error: Forbidden for url: https://sci-hub.ru/10.5586/asbp.1975.052
An exception occurred: 403 Client Error: Forbidden for url: https://sci-hub.ru/10.1007/BF00039808
An exception occurred: 403 Client Error: Forbidden for url: https://sci-hub.ru/10.2753/pet1061-1991200469
